In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.io
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from scipy.signal import welch
%matplotlib inline


In [ ]:
#fft_result = np.fft.fft(EEG2)
#freq = np.fft.fftfreq(len(EEG2), d=1/Fs)
#power_spectrum = np.abs(fft_result)
#plt.plot(freq[:len(freq)//2], power_spectrum[:len(power_spectrum)//2])
#plt.title("EEG Power Spectrum")
#plt.xlabel("Frequency (Hz)")
#plt.ylabel("Power")
#plt.yscale('log')
#plt.show()

In [ ]:
path= "/home/mario/code/SeanDominique/will-they-wake-up/data/physionet.org/files/i-care/2.1/training/1000/"

import os
scaler = StandardScaler()
def get_eeg_files(directory):
    eeg_files = []

    for root, _, files in os.walk(directory):

        for file in files:

            if file.endswith("EEG.mat"):

                absolute_path = os.path.abspath(os.path.join(root, file))
                eeg_files.append(absolute_path)

    return sorted(eeg_files)


EEG_file_list = get_eeg_files(path)
hours =[]
EEG_list = []
for file_path in EEG_file_list[:-1]:

    eeg = scipy.io.loadmat(file_path)
    eeg = eeg['val']
    eeg = eeg.astype(float)
    for line in eeg:
        temp_line = line.reshape(1, -1)
        temp_line = scaler.fit_transform(temp_line)
        line = temp_line.reshape(-1)
    eeg = np.mean(eeg,axis=0)
    EEG_list.append(eeg)

    hour = file_path[-11:-8]
    hours.append(hour)

len(hours)

In [ ]:
psds = []
for eeg in EEG_list:
    f, temp_psd = welch(eeg, fs=500, nperseg=1024)
    psds.append(temp_psd)


In [ ]:
fig = go.Figure()

index = 0
for psd in psds:
    color_intensity = min((1 - index/len(psds) ),1) # This is to change the color of each line. Plotly doesn't like to do it, so I had to force it with an f string and rgb values.
    fig.add_trace(go.Scatter(x = f, y = psd,
        mode='lines',

        line=dict( color= f"rgb({255*color_intensity},0,{255-(255*color_intensity)})"),

        opacity = 0.5,

    ))
    index += 1

fig.update_yaxes(type='log')
fig.show()
print(color_intensity)

In [ ]:
psds = np.array(psds)
psds.shape

In [ ]:
hours

In [ ]:
hours2 = pd.DataFrame(hours,columns=['hours'])
psds_df = pd.DataFrame(psds)
psds_df = pd.concat([psds_df, hours2], axis=1)
psds_df = psds_df.groupby(by='hours',as_index=True).mean()
psds_df = psds_df.div(psds_df[0],axis=0)

In [ ]:
psds_df[0]

In [ ]:
ini_psds = psds[:,:25]

ini_psds = ini_psds.T/(ini_psds[:,0])
ini_psds.shape

In [ ]:
px.line(psds_df,log_y=True,)

In [ ]:
ini_psds.T.shape

In [ ]:
EEG_file_list[50]